In [1]:
##NOTE: This notebook will run properly ONLY on PYTHON 3 KERNEL

import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
%matplotlib inline
import seaborn as sns
sns.set()
from matplotlib.backends.backend_pdf import PdfPages
#from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.colors as col
import matplotlib as mpl
import numpy as np #import for transposing
import math

In [2]:
obs_data = pd.read_csv('../Mongoose Modeling/Observations.csv')
obs_data = pd.DataFrame(obs_data)
obs_data.shape

(20851, 87)

# Necropsy Records


In [3]:
necrop = pd.read_csv('../Mongoose Modeling/Necropsy Records.csv')
necrop = necrop.rename(index = str, columns = {"CauseOfDeath":"COD"})
necrop.shape

(548, 39)

In [4]:
#Fixing isolated issues

##df.loc[row_index,column_name] <=>df[column_name][row_index]
##NOTE: when using .loc, row_index must be a string

necrop.loc['534','Month'] = 8
necrop.loc['534','Day'] = 15


for i in range(len(necrop)):
    if necrop.loc[str(i),'COD'] == 'Unknown':
        necrop.loc[str(i),'COD'] = 'Other'
        
    if necrop.loc[str(i),'COD'] == 'Self Predation':
        necrop.loc[str(i),'COD'] = 'Other'
        
    if str(i) in list(necrop.loc[necrop.COD.notnull() == False].index.values):
        necrop.loc[str(i),'COD'] = 'Other'


In [5]:
necrop["COD"].unique()

array(['TB', 'Other', 'Roadkill', 'Predation'], dtype=object)

In [6]:
##This function outputs an average number of mongoose killed by a particular cause per quarter/3 months
##NOTE: This does not give an actual percentage to be used in the model (see function: death_percent())
    
def death_rate(cause):
    ##Make a dataframe (df) with specified cause of death (cod)
    death = pd.DataFrame(necrop, columns = ["Year","Month","Day","COD"])
    death = death[death["COD"] == cause]
    cod = pd.DataFrame(death, columns = ["Year","Month","Day"]).dropna()
    cod = pd.DataFrame(pd.to_datetime(cod)).rename(index = str, columns = {0:"Date"}).sort_values(by='Date').reset_index(drop = True)

    #Add a column that strips years from the date
    #This makes sorting a little easier
    ser = []
    for i in range(len(cod)):
        ser.append(cod["Date"][i].year)
    cod["Year"] = ser  
    
    #Iterate through the unique years in the df
    #Each row of the "group" df represents a death/necropsy observation
    years = cod.Year.unique()
    dates = []
    rates = []
    for i in years:
        group = cod.groupby(['Year'])
        group = group.get_group(i).reset_index()
        if len(group) > 1 :
            date_diff = (group["Date"].iloc[-1]- group["Date"][0]).days
        
            if date_diff == 0:
                continue
            else:
                dates.append(date_diff)
                rate = len(group)/date_diff #The rates are mongoose per day
                rate = rate*(7*4*3) #This term comes from stoichiometry (puts the rate in the 3 month unit we need)
                rates.append(rate)
                total = sum(rates)/len(rates)
    for i in range(len(rates)):
        print(rates[i]," per 3 months")
    print()    
    print("Total average: ",total," per 3 months")
    
    return total

In [7]:
death_rate('Roadkill')

6.810810810810811  per 3 months
1.8529411764705883  per 3 months
1.9864864864864866  per 3 months
1.627906976744186  per 3 months
1.5319148936170213  per 3 months
2.638743455497382  per 3 months
5.813148788927336  per 3 months

Total average:  3.180278941221973  per 3 months


3.180278941221973

In [8]:
death_rate('TB')

3.888888888888889  per 3 months
2.2580645161290325  per 3 months
1.2844036697247707  per 3 months
2.3773584905660377  per 3 months
5.675675675675676  per 3 months
6.461538461538462  per 3 months

Total average:  3.657654950420478  per 3 months


3.657654950420478

In [9]:
death_rate('Predation')

2.8  per 3 months
2.2366863905325443  per 3 months
3.060728744939271  per 3 months
0.9940828402366864  per 3 months
0.8936170212765957  per 3 months
1.6258064516129034  per 3 months

Total average:  1.935153574766333  per 3 months


1.935153574766333

In [10]:
death_rate('Other')

7.875  per 3 months
5.478260869565217  per 3 months
19.384615384615387  per 3 months
17.50985915492958  per 3 months
5.7534246575342465  per 3 months
13.477611940298507  per 3 months
2.6782608695652175  per 3 months
4.085106382978723  per 3 months
2.3496503496503496  per 3 months
9.46075085324232  per 3 months
11.045871559633028  per 3 months
21.715909090909093  per 3 months
15.781818181818181  per 3 months

Total average:  10.507395330364602  per 3 months


10.507395330364602

# Necropsy Records Intersected with Census Data

Takes a look at the necropsy records with dates intersecting with the census data (standardized date.csv)

In [11]:
##"Importing" list of significant troops from the census data


mon_data = pd.read_csv("../Mongoose Modeling/Standardized Date.csv")

def clean_date(x):
        return datetime.strptime(x,'%d-%b-%y')

mon_data['Date'] = mon_data.Date.apply(lambda x: clean_date(x))

#################################################################

death = pd.DataFrame(mon_data, columns = ["Date","Troop","Troop Number","Complete Count?"])

death = death.dropna(subset = ["Troop Number","Date","Troop"])

death = death[death["Complete Count?"] != 'No']

#################################################################

##Additional df to sort death by troop

grouped = death.groupby("Troop")


#################################################################

wd = death.groupby("Troop").count().reset_index()

df = pd.DataFrame(wd, columns = ["Troop","Troop Number"])

#################################################################

clean = df[df["Troop Number"] > 10]

clean_list = list(clean["Troop"])

In [12]:
##troop_df contains necropsy records per troop
##death contains troop size info
    ##The names are a little backwards
troop_df = []

for i in clean_list:
    troop = necrop[necrop["Troop"] == i]
    copy = pd.DataFrame(troop, columns = ["Troop","COD"]).reset_index(drop = True)
    troop = pd.DataFrame(troop, columns = ["Year","Month","Day"])
    troop = pd.DataFrame(pd.to_datetime(troop)).rename(index = str, columns = {0:"Date"}).sort_values(by='Date').reset_index(drop = True)
    troop = pd.concat([troop,copy], axis = 1)
    
    troop_df.append(troop)


In [13]:
##Looking at troop_df[0] and grouped.get_group('CCH') to confirm it is doing what it is supposed to
troop_df[0]

,Date,Troop,COD
0,2000-06-29,CCH,TB
1,2000-07-10,CCH,Other
2,2000-08-13,CCH,Other
3,2003-06-01,CCH,TB
4,2004-09-16,CCH,TB
5,2007-12-12,CCH,Other
6,2007-12-22,CCH,Other
7,2007-12-27,CCH,Other
8,2008-01-08,CCH,Other
9,2008-01-08,CCH,Other


In [14]:
grouped.get_group('CCH')

,Date,Troop,Troop Number,Complete Count?
3,2015-08-27,CCH,11.0,NaN
37,2015-10-30,CCH,10.0,NaN
45,2015-11-21,CCH,13.0,NaN
51,2015-12-30,CCH,2.0,NaN
105,2016-01-18,CCH,8.0,NaN
115,2016-01-21,CCH,8.0,NaN
141,2016-01-29,CCH,1.0,NaN
144,2016-02-01,CCH,1.0,NaN
366,2016-04-27,CCH,33.0,NaN
594,2016-09-15,CCH,24.0,NaN


In [15]:
##cen_nec has the necropsy data within the dates from the census data
cen_nec = []

for i in range(len(clean_list)):
    wd = grouped.get_group(clean_list[i]).reset_index(drop = True)
    
    add = troop_df[i][troop_df[i]["Date"] >= wd["Date"][0]]
    add = add[add["Date"] <= wd["Date"][len(wd)-1]]
    
    cen_nec.append(add)
    

In [16]:
for i in range(len(clean_list)):
    print(i,"- ",clean_list[i]," has",len(cen_nec[i]),"observations")

0 -  CCH  has 6 observations
1 -  CGL  has 7 observations
2 -  CSL  has 21 observations
3 -  FOR  has 8 observations
4 -  KUBU  has 1 observations
5 -  LIB  has 3 observations
6 -  MOGO  has 2 observations
7 -  MOW  has 10 observations
8 -  PLAT  has 8 observations
9 -  SEF  has 14 observations
10 -  WA  has 11 observations
11 -  WDL  has 6 observations


# Necropsy Records Intersected with Observations Data 

In [17]:
##Not enough/conclusive data to get rates from census
##Attempt to extract census data from observation.csv

##Ignore troop membership

In [18]:
census = pd.DataFrame(obs_data, columns = ['ObservationTime','Troop'])

#fill na with 0 so that the sums aren't always na
census["Troop Size"] = obs_data['ST'].fillna(0) + obs_data['IT'].fillna(0)

#only want entries with actual troop sizes
census = census[census["Troop Size"] != 0].reset_index(drop = True).rename(index = str, columns = {'ObservationTime':'Date'})

census["Date"] = pd.to_datetime(census["Date"])

census


,Date,Troop,Troop Size
0,2008-05-06 10:10:00,CGL,34.0
1,2008-05-06 13:12:00,CCH,31.0
2,2008-05-16 11:25:00,MOW,13.0
3,2008-05-16 17:00:00,MOW,12.0
4,2008-05-17 10:30:00,TRS,16.0
5,2008-05-19 08:45:00,MOW,12.0
6,2008-05-19 09:53:00,CSL,18.0
7,2008-05-19 15:31:00,TRS,15.0
8,2008-05-20 07:00:00,MOW,12.0
9,2008-05-20 10:18:00,CSL,25.0


In [19]:
records = pd.DataFrame(necrop, columns = ["Year","Month","Day","COD"])
copy = records.copy().dropna().sort_values(by = ["Year","Month","Day"]).reset_index(drop = True)
records = pd.DataFrame(records, columns = ["Year","Month","Day"]).dropna()
records = pd.DataFrame(pd.to_datetime(records)).rename(index = str, columns = {0:"Date"}).sort_values(by='Date').reset_index(drop = True)
records = pd.concat([records,copy], axis = 1)

##Previous seemingly redundant steps are required to make sure the dates truly correspond to COD 
#(there are duplicate dates with different CODs)
#but we only need date and cod
records = records.drop(['Year','Month','Day'], axis = 1)

In [20]:
##sect contains necropsy data intersected with observation.csv (census) dates
#starting to run out of meaningful names

sect = records[records["Date"] >= census["Date"][0]]
sect = sect[sect["Date"] <= census["Date"][-1]].reset_index(drop = True)
sect.COD.unique()

array(['Other', 'TB', 'Predation', 'Roadkill'], dtype=object)

In [21]:
#Creates an actual percentage of particular causes of death per quarter to be used in model

def death_percent(cause):
    cod = sect[sect["COD"] == cause].reset_index(drop = True)
    
    
    #Add a column that strips years from the date
    #This makes sorting a little easier
    ser = []
    for i in range(len(cod)):
        ser.append(cod["Date"][i].year)
    cod["Year"] = ser 
    
    ##Add year column for troop number data (census)
    ser = []
    for i in range(len(census)):
        ser.append(census["Date"][i].year)
    census["Year"] = ser 
    
    
    #Iterate through the unique years in the df
    #Each row of the "group" df represents a death/necropsy observation
    years = cod.Year.unique()
    dates = []
    rates = []
    percent_list = []
    for i in years:
        group = cod.groupby(['Year'])
        group = group.get_group(i).reset_index()
        
        troop = census.groupby(['Year'])
        troop = troop.get_group(i).reset_index()
        
        
        
        if len(group) > 1 :
            date_diff = (group["Date"].iloc[-1]- group["Date"][0]).days
        
            if date_diff == 0:
                continue
            else:
                #This is here in case date differences need to be accessed
                dates.append(date_diff)
                
                #This is cleans the date column 
                troop["Date"] = troop["Date"].dt.date
                troop = troop.groupby(['Date','Troop']).max().reset_index()
                troop["Date"] = pd.to_datetime(troop["Date"])
                
                ##This loop gets the percentage of loss based on troop size around the date of death
                ## num deaths/troop size
                for k in range(len(group)): #iterate through group because each element is a death
                   
                    #This line finds the troop size for the nearest date to current element in group
                    num = troop["Troop Size"][troop["Date"] == min(troop["Date"],key=lambda date : abs(group["Date"][k]-date))].iloc[0]
            
                    percent = 1/num ##this is the case because every element in group represents one death 
                    percent_list.append(percent)
                    
                    percents = sum(percent_list)/len(percent_list)


                rate = percents/date_diff #The rates are mongoose per day
                rate = rate*(7*4*3) #This term comes from stoichiometry (puts the rate in the 3 month unit we need)
                rates.append(rate)
                total = sum(rates)/len(rates)
    for i in range(len(rates)):
        print(rates[i]," per 3 months")
    print()    
    print("Total average: ",total," per 3 months")
    
    
    return total    
    

In [22]:
death_percent('Roadkill')

0.0416648421556  per 3 months
0.0454678553592  per 3 months
0.066484951994  per 3 months

Total average:  0.0512058831696  per 3 months


0.051205883169561948

In [23]:
death_percent('Predation')

0.014527869791  per 3 months
0.0329541818994  per 3 months
0.0479036369008  per 3 months

Total average:  0.0317952295304  per 3 months


0.031795229530427065

In [24]:
death_percent('TB')

0.0326165660567  per 3 months
0.0397036874763  per 3 months
0.0259015972406  per 3 months
0.0454637964357  per 3 months

Total average:  0.0359214118023  per 3 months


0.035921411802331264

In [25]:
death_percent('Other')

0.0281235323388  per 3 months
0.021670981558  per 3 months
0.0275309021939  per 3 months
0.0346114543726  per 3 months

Total average:  0.0279842176158  per 3 months


0.027984217615829029